# Imports and file loading

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import numpy as np
from event import Event

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    f.close()
    return i + 1

def open_full_atf_file(file_name):
    file_length=file_len(file_name)
    file_handle=open(directory+file_name, 'r')
    csv_reader = csv.reader(file_handle, delimiter='\t', quotechar='|')
    data=np.empty((file_length,2))
    i=0
    for row in csv_reader:
        data[i,0]=row[0]
        data[i,1]=row[1]
        i+=1
    
    return data


def open_raw_file(file_name, data_point_start=-1, data_point_end=-1):
    # open full file
    if data_point_start == -1:
        data_point_start=0
    if data_point_end == -1:
        data_point_end=file_len(file_name)
    print data_point_end
    
    data=np.empty((data_point_end-data_point_start,2))
        
    file_handle=open(file_name, 'r')
    csv_reader = csv.reader(file_handle, delimiter='\t', quotechar='|')
    
    for i in range(0, data_point_start):
        row=csv_reader.next()
    
    for i in range(0, data_point_end-data_point_start):
        row=csv_reader.next()
        try:
            data[i,0]=row[0]
            data[i,1]=row[1]
        except:
            continue
            
    file_handle.close()
    
    return data

# Preliminary

In [ ]:
# File info
directory='./data/'
file_name='Chip4_HCT116_0021_0.005'
file_length=file_len(directory+file_name)

In [ ]:
# Define search parameters
interval=1000000

baseline_avg_length=100    # was 200
event_avg_length=3 # was 5
trigger_sigma_threshold=5
max_search_length=1000

event_indices=np.empty((0,2),dtype=int)
baseline=np.empty((0,4), dtype=float)

In [ ]:
i=0
start=1000
stop=interval
data=open_raw_file(directory+file_name, 1000, 1000000)

In [ ]:
baseline_avg_length=200    # was 200
event_avg_length=5
trigger_sigma_threshold=6
max_search_length=1000
baseline=np.empty((0,4))

current_index=0
while current_index+baseline_avg_length <= data.shape[0]:
    baseline_avg=np.mean(data[current_index:current_index+baseline_avg_length,1])
    baseline_sigma=np.std(data[current_index:current_index+baseline_avg_length,1])
    baseline=np.vstack((baseline, [[data[current_index,0], baseline_avg,\
                                    baseline_avg-1.*trigger_sigma_threshold*baseline_sigma,\
                                    baseline_avg+1.*trigger_sigma_threshold*baseline_sigma]]))
    current_index+=baseline_avg_length

In [ ]:
# Plot current over interval used to calculate the baseline
plt.plot(data[:baseline_avg_length,0], data[:baseline_avg_length,1])

plt.title('baseline interval')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel('time (s)')
plt.ylabel('current')

plt.show()

# Plot close up
a=0
b=2000
plt.plot(data[a:b,0], data[a:b,1])
#plt.plot(baseline[a:b,0], baseline[:,1], label='baseline')
plt.plot(baseline[:,0], baseline[:,2], label='trigger threshold (negative)', marker='o')
#plt.plot(baseline[a:b,0], baseline[:,3], label='trigger threshold (positive)')
plt.xlim(data[a,0],data[b,0])
plt.ylim(82000,86000)
plt.show()

# Plot baseline and trigger value
fig=plt.figure(figsize=(10,8))
plt.plot(data[:,0], data[:,1], label='data')
plt.plot(baseline[:,0], baseline[:,1], label='baseline')
plt.plot(baseline[:,0], baseline[:,2], label='trigger threshold (negative)')
plt.plot(baseline[:,0], baseline[:,3], label='trigger threshold (positive)')

plt.xlim(data[0,0], data[-1,0])

plt.legend(loc='lower right')

plt.title('data, baseline, trigger threshold')
plt.show()

# Begin file scan

In [ ]:
### baseline=np.empty((0,4), dtype=float)
event_indices=np.empty((0,2),dtype=int)

events=[]

i=0
file_finished=False

baseline=np.empty((0,4))

while i < 10:
    # Load next interval
    print 'i=',i
    start=1000+i*interval
    stop=start+interval
    if stop+interval > file_length:
        stop=file_length
        file_finished = True
    data=open_raw_file(directory+file_name, start, stop)
    i+=1
    
    
    try:
        current_index=0
        baseline_avg=np.mean(data[current_index:current_index+baseline_avg_length,1])
        baseline_sigma=np.std(data[current_index:current_index+baseline_avg_length,1])
        trigger_threshold=trigger_sigma_threshold*baseline_sigma
        baseline=np.vstack((baseline, np.array([[data[current_index,0], baseline_avg, baseline_avg-trigger_threshold,\
                                           baseline_avg+trigger_threshold]], dtype=float)))
        while True:
            # Look for events
            start_trigger_found = False
            while start_trigger_found == False:
                # Current exceeds trigger threshold
                if abs(data[current_index,1]-baseline_avg) >= trigger_threshold:
                    # Check to see which updated baseline to compare to
                    # Update baseline (i.e. in case of drift)
                    baseline_avg=np.mean(data[current_index-2*baseline_avg_length:current_index-baseline_avg_length,1])
                    baseline_sigma=np.std(data[current_index-2*baseline_avg_length:current_index-baseline_avg_length,1])
                    trigger_threshold=trigger_sigma_threshold*baseline_sigma
                    baseline=np.vstack((baseline, np.array([[data[current_index,0], baseline_avg, baseline_avg-1.*trigger_threshold,\
                                                baseline_avg+1.*trigger_threshold]], dtype=float)))
                            
                    # Point still exceeds trigger threshold
                    if abs(data[current_index,1]-baseline_avg) >= trigger_threshold:
                        # Trigger, get first point to exit baseline
                        event_avg=np.mean(data[current_index-event_avg_length:current_index,1])
                        start_index=current_index
                        
                        reentry_threshold=(baseline_avg+baseline_avg-trigger_threshold)/2.
                        while start_trigger_found == False:
                            if abs(data[start_index,1])>=abs(reentry_threshold):
                                print 'event #', event_indices.shape[0]
                                start_trigger_found = True
                            else:
                                event_avg=np.mean(data[start_index-event_avg_length:start_index,1])
                                start_index-=1
                    else:
                        continue
                current_index+=1
    
    
            # Look for possible event end
            stop_trigger_found = False
            while stop_trigger_found == False: 
                
                # Check if in baseline
                in_baseline=True
                for l in range(10):
                    if abs(data[current_index+l,1]-baseline_avg) >= trigger_threshold: 
                        in_baseline=False
                
                # Return to baseline
                if in_baseline:
                    stop_index=current_index
                    
                    # Find exact event end
                    event_avg=np.mean(data[current_index:current_index+event_avg_length,1])
                    
                    # Trigger, find first point to re-enter baseline
                    reentry_threshold=(baseline_avg+baseline_avg-trigger_threshold)/2.
                    while stop_trigger_found == False:
                        if abs(data[stop_index,1])>=abs(reentry_threshold):
                            stop_trigger_found = True
                        else:
                            stop_index+=1
                    
                    current_index=stop_index
                    event_indices=np.vstack((event_indices, [[start_index, stop_index+1]]))
                        
                    events.append(Event())
                    new_event=events[-1]
                    new_event.set_start_stop_index(1.*event_indices[-1,0], 1.*event_indices[-1,1])
                    new_event.set_data(data[event_indices[-1,0]:event_indices[-1,1],:])
                        
                    data[start_index:stop_index,1]=data[start_index-(stop_index-start_index):start_index,1]
    
                # Maximum search length
                if current_index-start_index >= max_search_length:
                    print 'hit maximum search length...\n'
                    print '\tcurrent_index=', current_index, 'start_index=', start_index, '\n'
                    stop_trigger_found = True
                    current_index=start_index+max_search_length #remove max search length
                    baseline_avg=np.mean(data[current_index:current_index+baseline_avg_length,1])
                    baseline_sigma=np.std(data[current_index:current_index+baseline_avg_length,1])
                    trigger_threshold=trigger_sigma_threshold*baseline_sigma
                    baseline=np.vstack((baseline, np.array([[data[current_index,0], baseline_avg, baseline_avg-1.*trigger_threshold,\
                                                   baseline_avg+1.*trigger_threshold]], dtype=float)))       
                    
                current_index+=1
                #print '\tt=', data[event_indices[-1,0],0]
                #print '\t', event_indices[-1,0], event_indices[-1,1], event_indices[-1,1]-event_indices[-1,0]
    except:
        continue

# Make plots

In [ ]:
plt.plot(data[:,0], data[:,1])
plt.plot(baseline[:,0], baseline[:,1])
plt.plot(baseline[:,0], baseline[:,2])
plt.xlim(data[0,0], data[1000000-1,0])
plt.show()

In [ ]:
delta=500
a=50
data=open_raw_file(directory+file_name, start, stop)
'''
for i in range(50):
    event=events[i]
    plt.plot(event._data[:,0], event._data[:,1], c=(1,0,0), label='Event '+str(i), lw=3)
    plt.plot(data[event._start_index-a:event._stop_index+a,0], data[event._start_index-a:event._stop_index+a,1])
    plt.plot(baseline[:,0], baseline[:,1], label='baseline')
    plt.plot(baseline[:,0], baseline[:,2], label='threshold')
    plt.xlim(plt.xlim(data[event._start_index-a,0], data[event._stop_index+a,0]))
    plt.ylim(event._data[:,1].min(), 1.001*event._data[:,1].max())
    

    
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.xlabel('time (s)')
    plt.ylabel(r'current ($\micro$A)')
    plt.legend(loc='lower left')
    plt.show()

# Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
for i in range(10):
    model=KMeans(n_clusters=3)
    events[i].set_predictions(model.fit_predict(events[i]._data[:,1].reshape(-1,1)).reshape(-1,1))
    events[i].set_levels(model.cluster_centers_)

In [ ]:
max_separation_length=10

In [ ]:
for i in range(10):
    event=events[i]
    level=event._predictions[0,0]
    for j in range(event._data.shape[0]):
        if event._predictions[j,0] == level:
            j=0
        else:
            j+=1
        if j == max_separation_length:
            level=event._predictions[j,0]
            event._split_list.append(j-max_separation_length)
            j=0

In [ ]:
for i in range(10):
    event=events[i]
    plt.scatter(event._data[:,0], event._data[:,1], c=event._predictions[:,0], zorder=10)


    for j in range(event._levels.shape[0]):
        plt.plot([-1000,1000],[event._levels[j], event._levels[j]], lw=5)

    for j in range(len(event._split_list)):
        time=event._data[event._split_list[j],0]
        plt.plot([time,time], [-1000000000,1000000000], ls='--', c=(.2,.2,.2))
    
    plt.title('event '+str(i))
    
    plt.xlim(event._data[0,0], event._data[-1,0])    
    plt.ylim(event._data[:,1].min(), event._data[:,1].max())
    plt.show()

# Amplitude duration scatter plot

In [ ]:
amplitudes=[]
durations=[]

for i in range(len(events)):
    event=events[i]
    event.calculate_amplitude()
    event.calculate_duration()
    
    amplitudes.append(event._amplitude)
    durations.append(event._duration)

In [ ]:
print amplitudes[:10]
print durations[:10]

In [ ]:
plt.scatter(durations, amplitudes)
plt.xlim(min(durations), max(durations)/2.)
plt.ylim(min(amplitudes), max(amplitudes))

plt.show()

In [ ]:
print len(events)